In [ ]:
%env CUPY_ACCELERATORS = cub
import sys
import unittest

import dask.array
import dask.distributed as dd
import cupy
import xarray as xr
# 
import src.geocat.comp.meteorology_gpu as geo_gpu
import src.geocat.comp.meteorology as geo

import numpy
import time
import pandas as pd

csvpath = "Test_meteorology.csv"

env: CUPY_ACCELERATORS=cub


In [ ]:
#reimplemented test functions for GPU/CPU benchmarking 
#Test_dewtemp
def test_list_input(xp = cupy):
    tk = (xp.asarray(t_def) + 273.15).tolist()
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.dewtemp_gpu(tk, rh_def) - 273.15, dt_2, 0.1)
    else:
        assert xp.allclose(geo.dewtemp(tk, rh_def) - 273.15, dt_2, 0.1)

def test_numpy_input(xp = cupy):
    tk = xp.asarray(t_def) + 273.15
    rh = xp.asarray(rh_def)
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.dewtemp_gpu(tk, rh) - 273.15, dt_2, 0.1)
    else:
        assert xp.allclose(geo.dewtemp(tk, rh) - 273.15, dt_2, 0.1)
        
def test_xarray_input(xp = cupy):
    tk = xr.DataArray(xp.asarray(t_def) + 273.15)
    rh = xr.DataArray(xp.asarray(rh_def))
    if(xp.__name__ == "cupy"):    
        assert xp.allclose(geo_gpu.dewtemp_gpu(tk, rh) - 273.15, dt_2, 0.1)
    else:
        assert xp.allclose(geo.dewtemp(tk, rh) - 273.15, dt_2, 0.1)

In [ ]:
#Test_dewtemp main body
Routine = "Test_dewtemp"
TestFunctionName = "test_list_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            t_def = xp.random.uniform(low=-52,high=29.5,size=ArraySize)
            dt_1 = 6.3
            rh_def = xp.random.uniform(low=41.7,high=90.5, size=ArraySize)
            tk = (xp.asarray(t_def) + 273.15).tolist()
            if(xp ==cupy):
                dt_2 = geo_gpu.dewtemp_gpu(tk, rh_def) - 273.15
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(dt_2),geo.dewtemp(xp.asnumpy(tk), xp.asnumpy(rh_def)) - 273.15): print("FALSE")
            else:
                dt_2 = geo.dewtemp(tk, rh_def) - 273.15
            test_list_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_list_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
#try:
#    previous = pd.read_csv(csvpath)
#    previous = pd.concat([previous,allData])
#except FileNotFoundError:
#    previous = allData
#previous.to_csv(csvpath, index=False)

In [ ]:
#Test_dewtemp main body
Routine = "Test_dewtemp"
TestFunctionName = "test_numpy_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            t_def = xp.random.uniform(low=-52,high=29.5,size=ArraySize)
            dt_1 = 6.3
            rh_def = xp.random.uniform(low=41.7,high=90.5, size=ArraySize)
            tk = (xp.asarray(t_def) + 273.15).tolist()
            if(xp ==cupy):
                dt_2 = geo_gpu.dewtemp_gpu(tk, rh_def) - 273.15
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(dt_2),geo.dewtemp(xp.asnumpy(tk), xp.asnumpy(rh_def)) - 273.15): print("FALSE")
            else:
                dt_2 = geo.dewtemp(tk, rh_def) - 273.15
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
#try:
#    previous = pd.read_csv(csvpath)
#    previous = pd.concat([previous,allData])
#except FileNotFoundError:
#    previous = allData
#previous.to_csv(csvpath, index=False)

In [ ]:
#Test_dewtemp main body
Routine = "Test_dewtemp"
TestFunctionName = "test_xarray_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,8):
    ArraySize = 10**i
    #print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            t_def = xp.random.uniform(low=-52,high=29.5,size=ArraySize)
            rh_def = xp.random.uniform(low=41.7,high=90.5, size=ArraySize)
            tk = (xp.asarray(t_def) + 273.15).tolist()
            if(xp ==cupy):
                dt_2 = geo_gpu.dewtemp_gpu(tk, rh_def) - 273.15
                #dt_2 = xr.DataArray(xp.asarray(dt_2))
                #validating GPU result with CPU results
                if not numpy.allclose(dt_2,geo.dewtemp(xp.asnumpy(tk), xp.asnumpy(rh_def)) - 273.15): print("FALSE")
            else:
                dt_2 = geo.dewtemp(tk, rh_def) - 273.15
            #test_xarray_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_xarray_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
#try:
#    previous = pd.read_csv(csvpath)
#    previous = pd.concat([previous,allData])
#except FileNotFoundError:
#    previous = allData
#previous.to_csv(csvpath, index=False)

In [ ]:
numpy.allclose(dt_2,geo.dewtemp(xp.asnumpy(tk), xp.asnumpy(rh_def)) - 273.15)

In [ ]:
geo_gpu.dewtemp_gpu(tk, rh)-273.15

In [ ]:
dt_2

In [ ]:
assert xp.allclose(geo_gpu.dewtemp_gpu(tk, rh) - 273.15, dt_2, 0.1)

In [ ]:
tk = xr.DataArray(xp.asarray(t_def) + 273.15)
rh = xr.DataArray(xp.asarray(rh_def))

In [ ]:
geo.dewtemp(xp.asnumpy(tk), xp.asnumpy(rh_def)) - 273.15

In [ ]:
type(dt_2)

In [ ]:
geo_gpu.dewtemp_gpu(tk, rh) - 273.15

In [ ]:
gc = 461.5  # gas constant for water vapor [j/{kg-k}]
gcx = gc / (1000 * 4.186)  # [cal/{g-k}]

lhv = (597.3 - 0.57 * (tk - 273)) / gcx
tdk = tk * lhv / (lhv - tk * numpy.log(rh * 0.01))
    

In [ ]:
a =  numpy.log(rh * 0.01)

In [ ]:
type(a.data)

In [ ]:
cupy.log(rh.data)

In [ ]:
cupy.asarray(rh)

In [ ]:
if(xp.__name__ == "cupy"):    
    assert xp.allclose(geo_gpu.dewtemp_gpu(tk, rh) - 273.15, dt_2, 0.1)
else:
    assert xp.allclose(geo.dewtemp(tk, rh) - 273.15, dt_2, 0.1)

In [ ]:
#reimplemented test functions for GPU/CPU benchmarking 
#Test_heat_index
def test_numpy_input(xp=cupy):
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.heat_index_gpu(t1, rh1, False),
                           ncl_gt_1,
                           atol=0.005)
    else:
        assert xp.allclose(geo.heat_index(t1, rh1, False),
                           ncl_gt_1,
                           atol=0.005)
def test_multi_dimensional_input(xp=cupy):
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.heat_index_gpu(t2.reshape(int(len(t2)/100), 100),
                           rh2.reshape(int(len(t2)/100), 100), True),
                           xp.asarray(ncl_gt_2).reshape(int(len(t2)/100), 100),
                           atol=0.005)
    else:
        assert xp.allclose(geo.heat_index(t2.reshape(int(len(t2)/100), 100),
                           rh2.reshape(int(len(t2)/100), 100), True),
                           xp.asarray(ncl_gt_2).reshape(int(len(t2)/100), 100),
                           atol=0.005)

def test_alt_coef(xp=cupy):
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.heat_index_gpu(t2, rh2, True),
                           ncl_gt_2,
                           atol=0.005)
    else:
        assert xp.allclose(geo.heat_index(t2, rh2, True),
                           ncl_gt_2,
                           atol=0.005)    

def test_list_input(xp=cupy):
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.heat_index_gpu(t1.tolist(), rh1.tolist()),
                           ncl_gt_1,
                           atol=0.005)
    else:
        assert xp.allclose(geo.heat_index(t1.tolist(), rh1.tolist()),
                           ncl_gt_1,
                           atol=0.005)

In [ ]:
#Test_heat_index main body
Routine = "Test_heat_index"
TestFunctionName = "_nws_eqn_gpu"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            t1 = xp.random.randint(low=30,high=104,size=10**i)
            rh1 = xp.random.randint(low=40,high=90,size=10**i)
            coeffs = [
            -42.379, 2.04901523, 10.14333127, -0.22475541, -0.00683783, -0.05481717,
            0.00122874, 0.00085282, -0.00000199
            ]
            coeffs = xp.asarray(coeffs)
            if(xp ==cupy):
                #validating GPU result with CPU results
                ncl_gt_1 = geo_gpu._nws_eqn_gpu(coeffs, t1, rh1)
                if not numpy.allclose(xp.asnumpy(ncl_gt_1),geo._nws_eqn(xp.asnumpy(coeffs), xp.asnumpy(t1),xp.asnumpy(rh1))): print("FALSE")
            else:
                ncl_gt_1 = geo._nws_eqn(t1, rh1, False)
            geo._nws_eqn(coeffs,t1,rh1)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            geo._nws_eqn(coeffs,t1,rh1)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
#try:
#    previous = pd.read_csv(csvpath)
#    previous = pd.concat([previous,allData])
#except FileNotFoundError:
#    previous = allData
#previous.to_csv(csvpath, index=False)

In [ ]:
#Test_heat_index main body
Routine = "Test_heat_index"
TestFunctionName = "test_numpy_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,6):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            t1 = xp.random.randint(low=30,high=104,size=10**i)
            rh1 = xp.random.randint(low=40,high=90,size=10**i)
            if(xp ==cupy):
                ncl_gt_1 = geo_gpu.heat_index_gpu(t1, rh1, False)
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(ncl_gt_1),geo.heat_index(xp.asnumpy(t1), xp.asnumpy(rh1), False)): print("FALSE")
            else:
                ncl_gt_1 = geo.heat_index(t1, rh1, False)
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
#try:
#    previous = pd.read_csv(csvpath)
#    previous = pd.concat([previous,allData])
#except FileNotFoundError:
#    previous = allData
#previous.to_csv(csvpath, index=False)

In [ ]:
#Test_heat_index main body
Routine = "Test_heat_index"
TestFunctionName = "test_multi_dimensional_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(2,6):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            t2 = xp.random.randint(low=70,high=115,size=10**i)
            rh2 = xp.random.randint(low=5,high=80,size=10**i)
            if(xp ==cupy):
                ncl_gt_2 = geo_gpu.heat_index_gpu(t2.reshape(int(len(t2)/100), 100),
                           rh2.reshape(int(len(t2)/100), 100), True)
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(ncl_gt_2),geo.heat_index(xp.asnumpy(t2.reshape(int(len(t2)/100), 100)),
                           xp.asnumpy(rh2.reshape(int(len(t2)/100), 100)), True)): print("FALSE")
            else:
                ncl_gt_2 = geo.heat_index(t2.reshape(int(len(t2)/100), 100),
                           rh2.reshape(int(len(t2)/100), 100), True)
            test_multi_dimensional_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_multi_dimensional_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
try:
    previous = pd.read_csv(csvpath)
    previous = pd.concat([previous,allData])
except FileNotFoundError:
    previous = allData
previous.to_csv(csvpath, index=False)

In [ ]:
#Test_heat_index main body
Routine = "Test_heat_index"
TestFunctionName = "test_alt_coef"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,6):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            t2 = xp.random.randint(low=70,high=115,size=10**i)
            rh2 = xp.random.randint(low=5,high=80,size=10**i)
            if(xp ==cupy):
                ncl_gt_2 = geo_gpu.heat_index_gpu(t2, rh2, True)
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(ncl_gt_2),geo.heat_index(xp.asnumpy(t2), xp.asnumpy(rh2), True)): print("FALSE")
            else:
                ncl_gt_2 = geo.heat_index(t2, rh2, True)
            test_alt_coef(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_alt_coef(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
try:
    previous = pd.read_csv(csvpath)
    previous = pd.concat([previous,allData])
except FileNotFoundError:
    previous = allData
previous.to_csv(csvpath, index=False)

In [ ]:
#Test_heat_index main body
Routine = "Test_heat_index"
TestFunctionName = "test_list_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,6):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            t1 = xp.random.randint(low=30,high=104,size=10**i)
            rh1 = xp.random.randint(low=40,high=90,size=10**i)
            if(xp ==cupy):
                ncl_gt_1 = geo_gpu.heat_index_gpu(t1, rh1, False)
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(ncl_gt_1),geo.heat_index(xp.asnumpy(t1), xp.asnumpy(rh1), False)): print("FALSE")
            else:
                ncl_gt_1 = geo.heat_index(t1, rh1, False)
            test_list_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_list_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
try:
    previous = pd.read_csv(csvpath)
    previous = pd.concat([previous,allData])
except FileNotFoundError:
    previous = allData
previous.to_csv(csvpath, index=False)

In [ ]:
#reimplemented test functions for GPU/CPU benchmarking 
#Test_relhum
def test_list_input(xp = cupy):
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.relhum_gpu(t_def, q_def, p_def),
                                      rh_gt_2,
                                      atol=0.1)
    else:
        assert xp.allclose(geo.relhum(t_def, q_def, p_def),
                                      rh_gt_2,
                                      atol=0.1)

def test_numpy_input(xp = cupy):
    p = xp.asarray(p_def)
    t = xp.asarray(t_def)
    q = xp.asarray(q_def)
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.relhum_gpu(t, q, p), rh_gt_2, atol=0.1)
    else:
        assert xp.allclose(geo.relhum(t, q, p), rh_gt_2, atol=0.1)

In [ ]:
#Test_relhum main body
Routine = "Test_relhum"
TestFunctionName = "test_list_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            p_def = xp.random.uniform(low=2000,high=100800,size=10**i)
            t_def = xp.random.uniform(low=194.65,high=302.45,size=10**i)
            q_def = xp.random.uniform(low=0,high=0.02038,size=10**i)
            if(xp ==cupy):
                rh_gt_2 = geo_gpu.relhum_gpu(t_def, q_def, p_def)
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(rh_gt_2),geo.relhum(xp.asnumpy(t_def), xp.asnumpy(q_def), xp.asnumpy(p_def))): print("FALSE")
            else:
                rh_gt_2 = geo.relhum(t_def, q_def, p_def)
            test_list_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_list_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
try:
    previous = pd.read_csv(csvpath)
    previous = pd.concat([previous,allData])
except FileNotFoundError:
    previous = allData
previous.to_csv(csvpath, index=False)

In [ ]:
#Test_relhum main body
Routine = "Test_relhum"
TestFunctionName = "test_numpy_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            p_def = xp.random.uniform(low=2000,high=100800,size=10**i)
            t_def = xp.random.uniform(low=194.65,high=302.45,size=10**i)
            q_def = xp.random.uniform(low=0,high=0.02038,size=10**i)
            if(xp ==cupy):
                rh_gt_2 = geo_gpu.relhum_gpu(t_def, q_def, p_def)
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(rh_gt_2),geo.relhum(xp.asnumpy(t_def), xp.asnumpy(q_def), xp.asnumpy(p_def))): print("FALSE")
            else:
                rh_gt_2 = geo.relhum(t_def, q_def, p_def)
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
try:
    previous = pd.read_csv(csvpath)
    previous = pd.concat([previous,allData])
except FileNotFoundError:
    previous = allData
previous.to_csv(csvpath, index=False)

In [ ]:
#implemented by my self 
#Test_relhum_water
rh_gt_1 = 46.3574
def test_numpy_input(xp):
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.relhum_water_gpu(t, q, p), rh_gt_1, atol=0.1)
    else:
        assert xp.allclose(geo.relhum_water(t, q, p), rh_gt_1, atol=0.1)

In [ ]:
#Test_relhum main body
Routine = "Test_relhum_water"
TestFunctionName = "test_numpy_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            p = xp.random.uniform(low=2000,high=100800,size=10**i)
            t = xp.random.uniform(low=194.65,high=302.45,size=10**i)
            q = xp.random.uniform(low=0,high=0.02038,size=10**i)
            if(xp ==cupy):
                rh_gt_1 = geo_gpu.relhum_water_gpu(t, q, p)
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(rh_gt_1),geo.relhum_water(xp.asnumpy(t), xp.asnumpy(q), xp.asnumpy(p))): print("FALSE")
            else:
                rh_gt_1 = geo.relhum_water(t, q, p)
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
try:
    previous = pd.read_csv(csvpath)
    previous = pd.concat([previous,allData])
except FileNotFoundError:
    previous = allData
previous.to_csv(csvpath, index=False)

In [ ]:
#implemented by my self 
#Test_relhum_ice
def test_numpy_input(xp = cupy):
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.relhum_ice_gpu(tk, w, p), rh_gt_1, atol=0.1)
    else:
        assert xp.allclose(geo.relhum_ice(tk, w, p), rh_gt_1, atol=0.1)

In [ ]:
#Test_relhum main body
Routine = "Test_relhum_ice"
TestFunctionName = "test_numpy_input"
print(Routine)
print(TestFunctionName)
allData = pd.DataFrame()
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in [numpy, cupy]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            p = xp.random.uniform(low=2000,high=100800,size=10**i)
            tc = xp.random.uniform(low=-10,high=0,size=10**i)
            w = xp.random.uniform(low=0,high=0.0037,size=10**i)
            tk = tc + 273.15
            if(xp ==cupy):
                rh_gt_1 = geo_gpu.relhum_ice_gpu(tk, w, p)
                #validating GPU result with CPU results
                if not numpy.allclose(xp.asnumpy(rh_gt_1),geo.relhum_ice(xp.asnumpy(tk), xp.asnumpy(w), xp.asnumpy(p))): print("FALSE")
            else:
                rh_gt_1 = geo.relhum_ice(tk, w, p)
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            test_numpy_input(xp)
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'Routine': numpy.repeat(Routine, repsize),
                'TestFunctionName': numpy.repeat(TestFunctionName, repsize),
                'Approach': numpy.repeat(xp.__name__ , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp.__name__,numpy.mean(repeat), "seconds") 
try:
    previous = pd.read_csv(csvpath)
    previous = pd.concat([previous,allData])
except FileNotFoundError:
    previous = allData
previous.to_csv(csvpath, index=False)